# Final Report - Team 2 NLP Project

Ella Xu, Jerry Nolf, Matthew Luna, Nathan Sharick - Innis Cohort

---

### Project Description:

- Most of the code hosting platforms for opensource projects consider the README file as the project introduction. As it is the first document seen by the reader, such a document needs to be crafted with care. The goal of this project is to predict the programming language for 100 repository by scraping, analyze the repository's README file contents. Using these datasets from 100 README's we were able to predict what programming language was used based on the composition of the README text.

### Project Goal:

- The goal of this project was to build a classification model that can predict the programming language of a repository based on the text of the repository's README.md file. 

### Imports

In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import unicodedata
import re
import json
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')
import matt_prepare

### Data Acquisition, Data Cleaning, and Data Preparation

- Web scraping methods were used to create a list of github username/repositories that included more than 300 repositories. The repositoriy names were pulled from github's top trending repositories, most forked repositories, and most stared repositories as of 05/13/ 2022.

- The list of repositories was put into the acompanying acquire.py file which creates a list of dictionaries that includes the name of the repository, the programming language used in the repository, and the content of the readme file for each repository in the list, and saves it as a .json file. The .json file is required to reproduce this project with this notebook and can be created by saving the acquire.py file in your local repository and running 'python acquire.py' from the terminal.

- Once the .json file is saved in the local directory is can be puled into the notebook, cleaned and prepared using the clean_df function located in the matt_prepare.py file. This function performs the following cleaning/preparation actions:
    - It uses pandas to read the file into the notebook
    
    - It cleans the data by normalizing it, changing all words to lower case, and removing any characters that are not letters, numbers, or whitespace
    
    - It tokenizes the words in the readme content using a toktok tokenizer
    
    - It removes standard english stopwords from the readme content
    
    - It then outputs a dataframe with the following columns:
        - name of the repository
        
        - Programming language of the repository
        
        - The raw readme contents
        
        - The cleaned readme content
        
        - The cleaned readme content that has been stemmed using a PorterStemmer
        
        - The cleaned readme content that has been lemmatized using a WordNetLemmatizer
        
        - The character count for the readme content
        
        - The word count for the readme content
        
- The dataset is then split into train, validate, and test sets using the split_data function located in the matt_prepare.py file which returns a train set with 56%, validate set with 24%, and test set with 20% of the original data frame


In [2]:
#pull in, clean, and prepare the dataset using the clean_df function
df = matt_prepare.clean_df()
#view the first five rows of the returned dataframe
df.head()

,repo,language,readme_contents,clean,stemmed,lemmatized,character_count,word_count
0,gocodeup/codeup-setup-script,Shell,# Codeup Setup Script\n\nSetup script for Code...,codeup setup script\n\nsetup script for codeu...,codeup setup script setup script for codeup st...,codeup setup script setup script for codeup st...,1395,203
1,gocodeup/movies-application,JavaScript,"# Movies Application\n\nFor this project, we w...",movies application\n\nfor this project we wil...,movi applic for thi project we will be build a...,movie application for this project we will be ...,4291,768
2,torvalds/linux,C,Linux kernel\n============\n\nThere are severa...,linux kernel\n\n\nthere are several guides for...,linux kernel there are sever guid for kernel d...,linux kernel there are several guide for kerne...,657,97
4,rdpeng/ProgrammingAssignment2,R,### Introduction\n\nThis second programming as...,introduction\n\nthis second programming assig...,introduct thi second program assign will requi...,introduction this second programming assignmen...,3554,620
5,octocat/Spoon-Knife,HTML,### Well hello there!\n\nThis repository is me...,well hello there\n\nthis repository is meant ...,well hello there thi repositori is meant to pr...,well hello there this repository is meant to p...,710,112


In [3]:
#split the dataset into train, validate, and test using the split_data function
train, validate, test = matt_prepare.split_data(df)
#view the row and column counts of the split dataframes
train.shape, validate.shape, test.shape

((150, 8), (65, 8), (54, 8))

---

### Data Exploration

- Data exploration comments.....

**Question 1:** What are the most common languages from the repos we explored?

**Question 2:** What are the most common words across all READme's?

**Question 3:** Does length of READme differ between languages?

**Question 4:** Are there any words that are found in all repos?

**Question 5:** ........

### Takeaways from Data Exploration

- Takeaway notes......

---

### Data Preparation for Modeling

- Notes on prep for modeling, bag of words, count vectorizer etc.......

In [4]:
#code for data prep for modeling

---

### Modeling

- Notes on modeling.....

In [6]:
#code for modeling

### Results from Modeling

- Notes on modeling results .....

---

### Summary

- Summary notes .....

### Next Steps

- Notes for next steps ....